## Описание проекта
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества F1 не меньше 0.75.

## Установка библиотек

In [1]:
# !pip install --upgrade scikit-learn --quiet

In [2]:
!pip install notifiers --quiet
!pip install nltk
!pip install lightgbm

In [3]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from lightgbm import LGBMClassifier

from sklearn.metrics import f1_score

import re
from notifiers import get_notifier

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sergf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sergf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sergf\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def make_notifier(token='1724537409:AAHuJxCLgdoMpP8c3StM4_sZj05DZ91T-BI',
                 chat_id=-521030020):
    def send(text):
        telegram=get_notifier('telegram')
        telegram.notify(message=text,
                       token=token,
                       chat_id=chat_id)
    return send
send = make_notifier()

## Загружаем массив

In [5]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [6]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [8]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Вывод: таблица имеет 2 колонки и 159571 строки, пропусков нет, токсичных комментариев примерно 10% от общего массива.

## Предобработка данных

In [9]:
nlp = WordNetLemmatizer()

Создадим функции обработки комментариев: будем удалять знаки препинания, символы переноса строки, после этого лемматизируем слова и соединим обратно в предложения.

In [10]:
def clear_text(row):
    txt = re.sub(r'[^a-zA-Z ]', " ", row)
    return " ".join(txt.split())

In [11]:
def lemm_text(row):
    row = clear_text(row)
    word_token = nltk.word_tokenize(row)
    return " ".join([nlp.lemmatize(word) for word in word_token])

In [12]:
df['lem_text'] = df['text'].apply(lemm_text)

In [13]:
df.head(20)

,text,toxic,lem_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...
5,"""\n\nCongratulations from me as well, use the ...",0,Congratulations from me a well use the tool we...
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7,Your vandalism to the Matt Shirvington article...,0,Your vandalism to the Matt Shirvington article...
8,Sorry if the word 'nonsense' was offensive to ...,0,Sorry if the word nonsense wa offensive to you...
9,alignment on this subject and which are contra...,0,alignment on this subject and which are contra...


## Базовая модель

Разделим модель на обучающую и тестовую выборки с сохранением соотношения токсичных и нетоксичных комментариев в выборках, применим TF-IDF векторизацию к обучающей и тестовой выборке по отдельности, чтобы оценить важность слов по отношению к массиву

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df['lem_text'], df['toxic'], test_size = 0.2, stratify = df['toxic'])

In [15]:
corpus_train = X_train.values.astype("U")
corpus_test = X_test.values.astype("U")

In [16]:
stopwords = set(nltk_stopwords.words('english'))
count_tfidf = TfidfVectorizer(stop_words=stopwords)
count_tfidf.fit(corpus_train)

TfidfVectorizer(stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

In [17]:
X_train_new = count_tfidf.transform(corpus_train)
X_train_new.shape

(127656, 144060)

Обучим модель Логистической регрессии на полученных данных для получения первичной оценки метрики F1, как референсной

In [18]:
clf = LogisticRegression().fit(X_train_new, y_train)

In [19]:
X_test_new = count_tfidf.transform(corpus_test)

In [20]:
pred = clf.predict(X_test_new)

In [21]:
f1_score(y_test, pred)

0.7333956969130028

Вывод: базовая модель дала результат F1 0.723, что ссужественно ниже, чем заявленный порог в 0.75, необходимо рассмотреть другие модели и подобрать гиперпараметры для них.

## Подбор моделей

Создадим таблицу-лог наших результатов

In [22]:
column = ['Модель', 'Параметры', 'Метрика F1']
results = pd.DataFrame(columns = column)

In [23]:
models = [
    LogisticRegression(),
    LGBMClassifier(),
    LinearSVC()
         ]

# Подбор гиперпараметров для моделей ниже занял часов 6, поэтому я оставил только лучшие показатели, 
# но изменить список значений на range(start, stop, step) не составит сложности в случае необходимости.

params = [
    {
        'model__penalty':['l1'],
        'model__C':[1],
        'model__solver':['liblinear'],
        'model__random_state':[0]
        },
    {
        'model__max_depth':[19],
        'model__learning_rate' : [0.5],
        'model__n_estimators':[170],
        'model__random_state':[0]
        },
    {
        'model__penalty':['l2'],
        'model__C':[0.5],
        'model__loss':['squared_hinge'],
        'model__random_state':[0],
    }
]

In [24]:
for model, param in zip(models, params):
    pipe = Pipeline(steps = [('ti_idf', TfidfVectorizer(stop_words = set(nltk_stopwords.words('english')))),
                   ('model', model)])
    grid = GridSearchCV(estimator = pipe, param_grid = param, n_jobs =-1, scoring = 'f1', verbose = 4).fit(corpus_train, y_train)
    f1_scor = f1_score(y_test, grid.best_estimator_.predict(corpus_test))
    results = pd.concat([results, pd.DataFrame([[model, grid.best_params_, f1_scor]], columns = column)])

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [25]:
results

,Модель,Параметры,Метрика F1
0,LogisticRegression(),"{'model__C': 1, 'model__penalty': 'l1', 'model...",0.771110
0,LGBMClassifier(),"{'model__learning_rate': 0.5, 'model__max_dept...",0.768763
0,LinearSVC(),"{'model__C': 0.5, 'model__loss': 'squared_hing...",0.775947


In [26]:
results[results['Метрика F1']==results['Метрика F1'].max()]

,Модель,Параметры,Метрика F1
0,LinearSVC(),"{'model__C': 0.5, 'model__loss': 'squared_hing...",0.775947


In [27]:
send(f"Код выполнен, лучший результат: {results[results['Метрика F1']==results['Метрика F1'].max()]}")

Вывод: мы провели анализ массива на токсичные комментарии пользователей, обучили модели предсказывать такие комментарии, использовали метрику гармонического среднего - F1, лучший результат дала модель опорных векторов LinearSVC, метрика F1 составила 0.776